## 2유형 [3] Census Income

In [34]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
import numpy as np
X_test = pd.read_csv("./datasets/Part2/census_X_test.csv")
X_train = pd.read_csv("./datasets/Part2/census_X_train.csv")
y_train = pd.read_csv("./datasets/Part2/census_y_train.csv")

# 사용자 코딩

### 1. 데이터 전처리
# 1) 데이터 확인 -> 결측치는 따로 없음
# print(X_train.info())

# 2) 기술통계량
# print(X_train.describe())

# 3) 기술통계량에서 capital_gain, capital_loss 이상치 발견 (오른쪽으로 치우침) => 백분위수 쪼개서 살펴보기
# print(X_train['capital_gain'].quantile([q/20 for q in range(15, 21)]))
# print(X_train['capital_loss'].quantile([q/20 for q in range(15, 21)]))

# 4) 범주형 변수로 바꾸어 파생변수를 생성하고, 기존변수와 중요도 확인 (더 높은 것 사용)
X_train['capital_gain_yn'] = np.where(X_train['capital_gain']>0, 1, 0)
X_train['capital_loss_yn'] = np.where(X_train['capital_loss']>0, 1, 0)

X_test['capital_gain_yn'] = np.where(X_test['capital_gain']>0, 1, 0)
X_test['capital_loss_yn'] = np.where(X_test['capital_loss']>0, 1, 0)

# 5) 종속변수 값별로 각 독립변수의 분포 확인 (숫자형)
# 컬럼명 저장
COL_DEL = []
COL_NUM = ['age', 'education_num', 'hours_per_week', 'capital_gain', 'capital_loss']
COL_CAT = ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']
COL_Y = ['target']

X_train = X_train.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

# 기술통계량 확인
# 종속변수의 값이 1일 때의 평균이 0일 때의 평균보다 크므로 예측에 사용할 수 있는 데이터 확인
train_df = pd.concat([X_train, y_train], axis=1)

# for col in COL_NUM:
#     print('-' * 80)
#     print(col)
#     print(train_df.groupby(COL_Y)[col].describe(), end='\n\n')

# 6) 각 범주형 변수별로 종속변수의 평균을 확인 (문자형)
# for col in COL_CAT:
#     print(col)
#     print(train_df.groupby(col, as_index=False)[COL_Y].mean().sort_values(by=COL_Y, ascending=False), end='\n\n')

# 7) 레이블 인코딩 (범주형 변수들 -> 숫자로 변경)
from sklearn.preprocessing import LabelEncoder

X = pd.concat([X_train, X_test])
for col in COL_CAT:
    le = LabelEncoder()
    le.fit(X_train[col])
    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

### --------------------

### 2. 모델 구축
# 1) 데이터 분리
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)

# 2) 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr[COL_NUM] = scaler.fit_transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.fit_transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.fit_transform(X_test[COL_NUM])

# 3) 분류모델 사용: 랜덤포레스트, XGBoost
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.ravel())

from xgboost import XGBClassifier

# n_estimators=100
model_xgb1 = XGBClassifier()
model_xgb1.fit(X_tr, y_tr.values.ravel())

# XGBoost 학습 시, eval_set을 사용하여 과적합 방지 가능
# early_stopping_rounds 이상의 iteration 동안 성능 개선이 없으면 학습을 멈춤
# model_xgb2 = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=10)
# model_xgb2.fit(X_tr, y_tr.values.ravel(), early_stopping_rounds=50, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=10)

### --------------------

### 3. 모델 평가 (ROC_AUC_SCORE) & 하이퍼 파라미터 튜닝
# 1) ROC-AUC 값 구하기
from sklearn.metrics import roc_auc_score

y_pred_rf = model_rf.predict_proba(X_val)
y_pred_xgb1 = model_xgb1.predict_proba(X_val)

score_rf = roc_auc_score(y_val, y_pred_rf[:, 1])
score_xgb1 = roc_auc_score(y_val, y_pred_xgb1[:, 1])

# print(score_rf) : 0.8894481775603122
# print(score_xgb1) : 0.9123917379702831 -> XGBoost가 성능이 더 좋음

# 2) 하이퍼 파라미터 튜닝 전, 파생변수의 변수 중요도 확인
# print(pd.DataFrame({'feature': X_tr.columns, 'fi_rf': model_rf.feature_importances_, 
#                     'fi_xgb': model_xgb1.feature_importances_}))

# 중요도가 너무 낮으므로 해당 컬럼 제외
COL_DEL = ['capital_gain_yn', 'capital_loss_yn']

X_tr = X_tr.drop(COL_DEL, axis=1)
X_val = X_val.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

# 3) 하이퍼 파라미터 튜닝 (랜덤 포레스트)
# from sklearn.model_selection import GridSearchCV

# grid_params = { 
#     'max_depth': [5, 10, 15],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# rf_cv = GridSearchCV(estimator=model_rf, param_grid=grid_params, cv=5)
# rf_cv.fit(X_train, y_train.values.ravel()) 

# print(pd.DataFrame(rf_cv.cv_results_).head())
# print(rf_cv.best_params_)

# 4) 하이퍼 파라미터 튜닝 (XGBoost)
# model_rf2 = RandomForestClassifier(n_estimators=50
#                                    , max_depth=15
#                                    , min_samples_leaf=1
#                                    , min_samples_split=5)
# model_rf2.fit(X_tr, y_tr.values.ravel())

# y_pred_rf2 = model_rf2.predict_proba(X_val)
# score_rf2 = roc_auc_score(y_val, y_pred_rf2[:, 1])
# print(score_rf2)

# grid_params = {'max_depth': [3, 5, 7, 10], 
#                'min_child_weight': [1, 2], 
#                'colsample_bytree': [0.6, 0.8],
#                'subsample': [0.6, 0.8]}

# xgb_cv = GridSearchCV(estimator=model_xgb1, param_grid=grid_params, cv=5)
# xgb_cv.fit(X_tr, y_tr.values.ravel())

# print(xgb_cv.best_params_)

# 5) 가장 좋은 하이퍼 파라미터로 재학습
params = {'colsample_bytree': 0.6,
          'max_depth': 7,
          'min_child_weight': 1,
          'subsample': 0.8}

model_xgb3 = XGBClassifier(n_estimators=1000, learning_rate=0.05)
model_xgb3.set_params(**params)

model_xgb3.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=10)

# print(model_xgb3.best_score)

### --------------------

### 4. 제출 파일로 변환

# 1) 모델 예측 값 계산
pred = model_xgb3.predict_proba(X_test)[:, 1]

# 2) 파일 저장
pd.DataFrame({'index': X_test.index, 'target': pred}).to_csv('./res/003000000.csv', index=False)

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)

[0]	validation_0-auc:0.86711
[10]	validation_0-auc:0.90872
[20]	validation_0-auc:0.91014


C:\Users\phi49\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\phi49\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[30]	validation_0-auc:0.91107
[40]	validation_0-auc:0.91134
[50]	validation_0-auc:0.91115
[60]	validation_0-auc:0.91123
[70]	validation_0-auc:0.91157
[80]	validation_0-auc:0.91159
[90]	validation_0-auc:0.91160
[100]	validation_0-auc:0.91155
[110]	validation_0-auc:0.91197
[120]	validation_0-auc:0.91175
[130]	validation_0-auc:0.91179
[140]	validation_0-auc:0.91176
[150]	validation_0-auc:0.91171
[158]	validation_0-auc:0.91170
